In [30]:
import pyate
import pandas as pd
import numpy as np
import scattertext as st
import spacy
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import sys
import pyate

from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

tqdm.pandas()
sys.path.append('../bingocardgenerator')

/Users/jasonkessler/anaconda3/envs/py38/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [34]:
assert st.version >= [0, 0, 2, 69]

In [2]:
def parse_url(url):
    data = []
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    try:
        for p in soup.find('div', attrs={'class': 'fl-callout-content'}).findAll('p'):
            raw = p.text.strip()
            if raw == '': continue
            content = '\n'.join(raw.split('\n')[1:])
            speaker = raw.split('\n')[0].split(':')[0]
            time = raw.split('\n')[0].split(':', 1)[1].strip()
            time = time[1:-1]
            if len(time) == 5: 
                time = '00:' + time
            data.append({'Speaker': speaker, 'Time': time, 'Text': content, 'URL': url})
    except:
        print(f'Cannot parse {url}')
    return data

In [3]:
harris_urls = [
    'https://www.rev.com/blog/transcripts/kamala-harris-voter-mobilization-event-speech-transcript-in-nevada-october-2',
    'https://www.rev.com/blog/transcripts/kamala-harris-campaign-roundtable-transcript-in-north-carolina-september-28',
    'https://www.rev.com/blog/transcripts/kamala-harris-speech-transcript-on-trumps-scotus-nominee-the-2020-race-september-28',
    'https://www.rev.com/blog/transcripts/naacp-national-convention-with-kamala-harris-transcript-september-25',
    'https://www.rev.com/blog/transcripts/kamala-harris-shop-talk-campaign-visit-detroit-michigan-transcript-september-22',
    'https://www.rev.com/blog/transcripts/democratic-debate-transcript-las-vegas-nevada-debate',
    'https://www.rev.com/blog/transcripts/new-hampshire-democratic-debate-transcript',
    'https://www.rev.com/blog/transcripts/january-iowa-democratic-debate-transcript',
    'https://www.rev.com/blog/transcripts/transcript-of-july-democratic-debate-2nd-round-night-2-full-transcript-july-31-2019',
    'https://www.rev.com/blog/transcripts/democratic-debate-transcript-houston-september-12-2019',
    'https://www.rev.com/blog/transcripts/october-democratic-debate-transcript-4th-debate-from-ohio',
    'https://www.rev.com/blog/transcripts/november-democratic-debate-transcript-atlanta-debate-transcript',
    'https://www.rev.com/blog/transcripts/december-democratic-debate-transcript-sixth-debate-from-los-angeles',
    'https://www.rev.com/blog/transcripts/transcript-of-july-democratic-debate-night-1-full-transcript-july-30-2019'
    
]
data = []
for url in harris_urls:
    data += parse_url(url)

Cannot parse https://www.rev.com/blog/transcripts/january-iowa-democratic-debate-transcript
Cannot parse https://www.rev.com/blog/transcripts/transcript-of-july-democratic-debate-night-1-full-transcript-july-30-2019


In [4]:
pence_urls = [
    'https://www.rev.com/blog/transcripts/mike-pence-faith-in-leadership-speech-transcript-october-1',
    'https://www.rev.com/blog/transcripts/mike-pence-carter-lake-iowa-campaign-visit-speech-transcript-october-1',
    'https://www.rev.com/blog/transcripts/mike-pence-gilford-nh-campaign-speech-transcript-september-22',
    'https://www.rev.com/blog/transcripts/mike-pence-coronavirus-safety-in-nursing-homes-roundtable-transcript-september-17',
    'https://www.rev.com/blog/transcripts/mike-pence-workers-for-trump-event-speech-transcript-zanesville-ohio-september-16',
    'https://www.rev.com/blog/transcripts/mike-pence-campaign-speech-transcript-janesville-wisconsin-september-14',
    'https://www.rev.com/blog/transcripts/mike-pence-speech-at-vmi-transcript-september-10',
    'https://www.rev.com/blog/transcripts/mike-pence-speech-right-to-life-event-transcript-september-3',
    'https://www.rev.com/blog/transcripts/mike-pence-workers-for-trump-event-speech-transcript-september-1'
    
]
for url in pence_urls:
    data += parse_url(url)

In [5]:
vpdebate_2016 = open('../data/vpdebate_2016.txt').read()
speaker = None
debatedata = []
for line in vpdebate_2016.strip().split('\n'):
    if not line.strip(): continue
    if line.startswith('PENCE:') or line.startswith('KAINE:') or line.startswith('QUIJANO'):
        speaker = line.split(':')[0]
        line = line.split(':', 1)[1]
    if speaker == 'PENCE':
        debatedata.append({'Speaker': 'PENCE', 'Time': None, 'Text': line.strip(), 'URL': 'https://www.debates.org/voter-education/debate-transcripts/october-4-2016-debate-transcript/'})
data += debatedata

In [10]:
df = pd.DataFrame(data)
pence_harris_df = df[
    df.Speaker.str.lower().apply(lambda x: 'pence' in x or 'harris' in x)
].assign(
    Name = lambda df: df.Speaker.str.lower().apply(
        lambda x: 'Pence' if 'pence' in x else 'Harris'
    ),
    Parse = lambda df: df.Text.apply(st.whitespace_nlp_with_sentences)
)

In [11]:
corpus = st.CorpusFromParsedDocuments(
    pence_harris_df, 
    parsed_col='Parse', 
    category_col='Name', 
    feats_from_spacy_doc=st.PyatePhrases(pyate.combo_basic)
).build()

In [27]:
# Now let's make the bingo card

from square_getter import SquareGetter
from card_maker import write_card

In [43]:
bingo_card_squares = SquareGetter(corpus).generate_squares(shuffle=False)
bingo_card_squares[0] = bingo_card_squares[12]
bingo_card_squares[12] = 'Free Space: Pence: four more years' # The free space
bingo_card_squares

/Users/jasonkessler/anaconda3/envs/py38/lib/python3.7/site-packages/scipy-1.4.1-py3.7-macosx-10.9-x86_64.egg/scipy/sparse/base.py:599: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)
/Users/jasonkessler/anaconda3/envs/py38/lib/python3.7/site-packages/scipy-1.4.1-py3.7-macosx-10.9-x86_64.egg/scipy/sparse/base.py:599: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


['Pence: american energy',
 'Harris: criminal justice system',
 'Pence: law enforcement',
 'Harris: justice system',
 'Pence: american people',
 'Harris: criminal justice',
 'Pence: american history',
 'Harris: civil rights',
 'Pence: armed forces',
 'Harris: public schools',
 'Pence: major cities',
 'Harris: home ownership',
 'Free Space: Pence: four more years',
 'Harris: excessive force',
 'Pence: short years',
 'Harris: safe communities',
 'Pence: american workers',
 'Harris: black women',
 'Pence: federal red tape',
 'Harris: mental health care',
 'Pence: radical left',
 'Harris: black woman',
 'Pence: fair trade',
 'Harris: public health',
 'Pence: great depression']

In [54]:
html = write_card(card_title = 'Automatically Created 2020 VP Debate Bingo Card',
                  card_subtitle = '',
                  squares=bingo_card_squares)


In [55]:
fn = 'vpdebate2020.html'
open(fn, 'w').write(html)
IFrame(src=fn, width = 1300, height=700)
